# Key Rotation

Vault uses unseal keys for the following operations:

- generate root tokens
- seal the Vault
- unseal the Vault

As these operations are highly sensitive, it is very important to keep the keys very secure. One way to ensure the security of these keys, is to rotate them regularly. Moreover, it might be necessary to rotate them due to a leak, or because new key shares are required from a joiner/mover/leaver process. In this tutorial we will learn to work with these keys.

### Starting Vault

Let us start a Vault server. This will run Vault in the background and push the logs to `/tmp/vault.log`. If at any point in time the Vault crashes, this command will need to be used again to re-launch the Vault server.

In [ ]:
nohup bash -c '
  vault server -config ./assets/shamir-config.hcl
' > /tmp/vault.log 2>&1 &
echo $! > /tmp/vault.pid

### Generating a Set of Unseal Keys

We will generate a first set of unseal keys for our Vault. We will use 5 key shares and a threshold of 3.

In [ ]:
export VAULT_ADDR="http://localhost:8200"
vault operator init -format=json | tee /tmp/vault-unseal.json

In [ ]:
# unseal the vault
vault operator unseal "$(jq -r '.unseal_keys_b64[0]' < /tmp/vault-unseal.json)" > /dev/null
vault operator unseal "$(jq -r '.unseal_keys_b64[1]' < /tmp/vault-unseal.json)" > /dev/null
vault operator unseal "$(jq -r '.unseal_keys_b64[2]' < /tmp/vault-unseal.json)" > /dev/null

### Simple Rotations

Simple rotations do not involve any additional layer of security. In other words, the keys are rotated by initialising a rotation, and then providing the current unseal keys to Vault. Once this is done, the new keys are returned. This is typically not ideal, as the person providing the last old key share will see all new keys and need to distribute them. In practice you would want to request Vault to GPG encrypt these key shares first, so that the person entering the last key share gets encrypted data as a response.

Let us start a simple rekey. Update the command below to create 7 key shares (assume the team managing Vault got two new members), with a key threshold still of 3.

In [ ]:
vault operator rekey -init -key-shares=<x> -key-threshold=<y> -format=json | tee /tmp/vault-rekey.json

In [ ]:
# check the status of the current rekey operation
vault operator rekey -status

In [ ]:
# provide first unseal key share
vault operator rekey -nonce="$(jq -r '.nonce' < /tmp/vault-rekey.json)" "$(jq -r '.unseal_keys_b64[0]' < /tmp/vault-unseal.json)"

In [ ]:
# provide second unseal key share
vault operator rekey -nonce="$(jq -r '.nonce' < /tmp/vault-rekey.json)" "$(jq -r '.unseal_keys_b64[1]' < /tmp/vault-unseal.json)" > /dev/null

In [ ]:
# provide third and last unseal key share
vault operator rekey -format=json -nonce="$(jq -r '.nonce' < /tmp/vault-rekey.json)" "$(jq -r '.unseal_keys_b64[2]' < /tmp/vault-unseal.json)"| tee /tmp/vault-rekey-result.json

## Cleaning Up

At the end of each module, you should clean up your Vault instance. This is done by shutting it down and wiping its database to restore its state.

In [ ]:
kill $(cat /tmp/vault.pid)
rm /tmp/vault/vault.db
rm -r /tmp/vault/raft/
rm /tmp/vault.log
rm /tmp/vault.pid
rm /tmp/vault-unseal.json
rm /tmp/vault-rekey.json
rm /tmp/vault-rekey-result.json